In [1]:
import pydantic

In [2]:
pydantic.__version__

'2.4.2'

# 2. 클래스 선언해서 처리하기 

In [30]:
from datetime import datetime
from typing import Tuple, List, Optional

from pydantic import BaseModel, PositiveInt, ValidationError, field_validator

## 2-1 pydantic에서 객체를 정의하는 것은 BaseModel에서 상속되는 새 클래스를 만드는 것

In [18]:
class Delivery(BaseModel):
    timestamp: datetime
    dimensions: Tuple[int, int]

In [19]:
m = Delivery(timestamp='2020-01-02T03:04:05Z', dimensions=['10', '20'])

In [20]:
m.timestamp

datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))

In [21]:
print(repr(m.timestamp))

datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))


In [22]:
print(m.dimensions)

(10, 20)


## 2-2. 클래스 선언해서 처리하기 

## 초기값을 지정한 클래스 선언 

In [23]:
class User(BaseModel):
    id: int  
    name: str = 'John Doe'                ## 초기값 지정 
    signup_ts: datetime | None            ## 유니언타입 지정 
    tastes: dict[str, PositiveInt]        ## 양수처리 타입 지정 

In [24]:
external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'tastes': {
        'wine': 9,
        b'cheese': 7,  
        'cabbage': '1',  
    },
}

In [25]:
user = User(**external_data) 

In [26]:
user.__dict__

{'id': 123,
 'name': 'John Doe',
 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}

## 3. 검증처리 

In [27]:
external_data2 = {'id': 'not an int', 'tastes': {}}  


In [28]:
try:
    User(**external_data2)  
except ValidationError as e:
    print(e.errors())

[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'not an int', 'url': 'https://errors.pydantic.dev/2.4/v/int_parsing'}, {'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 'not an int', 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.4/v/missing'}]


## 특정 필드 검증하기

### Validator
- 또한 상속된 클래스 내부의 유효성 검사기 데코레이터를 사용하여 고유한 사용자 지정 유효성 검사기를 만들 수 있습니다.

- id가 4자리인지 확인하고 Confirm_password가 비밀번호 필드와 일치하는지 확인하는 다음 예를 살펴보겠습니다.

In [35]:
class User2(BaseModel):
    id: int
    username : str
    password : str
    confirm_password : str
    timestamp: Optional[datetime] = None
    friends: List[int] = []
    @field_validator('id')
    def id_must_be_4_digits(cls, v):
        if len(str(v)) != 4:
            raise ValueError('must be 4 digits')
        return v


In [36]:
data2 = {'id': '1234', 'username': 'wai foong', 'password': 'Password123', 
        'confirm_password': 'Password123', 
        'timestamp': '2020-08-03 10:30', 'friends': [1, '2', b'3']}

In [37]:
user2 = User2(**data2)

In [38]:
user2.__dict__

{'id': 1234,
 'username': 'wai foong',
 'password': 'Password123',
 'confirm_password': 'Password123',
 'timestamp': datetime.datetime(2020, 8, 3, 10, 30),
 'friends': [1, 2, 3]}

### 검증결과 에러 처리 

In [39]:
data3 = {'id': '123456', 'username': 'wai foong', 'password': 'Password123', 
        'confirm_password': 'Password123', 
        'timestamp': '2020-08-03 10:30', 'friends': [1, '2', b'3']}

In [40]:
user3 = User2(**data3)

ValidationError: 1 validation error for User2
id
  Value error, must be 4 digits [type=value_error, input_value='123456', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error

## 4. 클래스 내의 메서드를 확인하기 

In [42]:
class User1(BaseModel):
    id: int
    username : str
    password : str
    confirm_password : str
    timestamp: Optional[datetime] = None
    friends: List[int] = []

In [43]:
data = {'id': '1234', 'username': 'wai foong', 'password': 'Password123', 
        'confirm_password': 'Password123', 
        'timestamp': '2020-08-03 10:30', 'friends': [1, '2', b'3']}

In [44]:
user = User1(**data)

### 객체 내의 값 확인하기 

In [45]:
user.__dict__

{'id': 1234,
 'username': 'wai foong',
 'password': 'Password123',
 'confirm_password': 'Password123',
 'timestamp': datetime.datetime(2020, 8, 3, 10, 30),
 'friends': [1, 2, 3]}

In [46]:
user.dict()

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_74362/336500289.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  user.dict()


{'id': 1234,
 'username': 'wai foong',
 'password': 'Password123',
 'confirm_password': 'Password123',
 'timestamp': datetime.datetime(2020, 8, 3, 10, 30),
 'friends': [1, 2, 3]}

In [52]:
user.model_dump()

{'id': 1234,
 'username': 'wai foong',
 'password': 'Password123',
 'confirm_password': 'Password123',
 'timestamp': datetime.datetime(2020, 8, 3, 10, 30),
 'friends': [1, 2, 3]}

### json으로 확인하기 

In [47]:
user.json()

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_74362/3609156729.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  user.json()


'{"id":1234,"username":"wai foong","password":"Password123","confirm_password":"Password123","timestamp":"2020-08-03T10:30:00","friends":[1,2,3]}'

In [51]:
user.model_dump_json()

'{"id":1234,"username":"wai foong","password":"Password123","confirm_password":"Password123","timestamp":"2020-08-03T10:30:00","friends":[1,2,3]}'

### 데이터의 스키마 정보 확인하기 

In [48]:
user.schema()

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_74362/2470975238.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  user.schema()


{'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'username': {'title': 'Username', 'type': 'string'},
  'password': {'title': 'Password', 'type': 'string'},
  'confirm_password': {'title': 'Confirm Password', 'type': 'string'},
  'timestamp': {'anyOf': [{'format': 'date-time', 'type': 'string'},
    {'type': 'null'}],
   'default': None,
   'title': 'Timestamp'},
  'friends': {'default': [],
   'items': {'type': 'integer'},
   'title': 'Friends',
   'type': 'array'}},
 'required': ['id', 'username', 'password', 'confirm_password'],
 'title': 'User1',
 'type': 'object'}

In [49]:
user.schema_json()

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_74362/59920019.py:1: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  user.schema_json()


'{"properties": {"id": {"title": "Id", "type": "integer"}, "username": {"title": "Username", "type": "string"}, "password": {"title": "Password", "type": "string"}, "confirm_password": {"title": "Confirm Password", "type": "string"}, "timestamp": {"anyOf": [{"format": "date-time", "type": "string"}, {"type": "null"}], "default": null, "title": "Timestamp"}, "friends": {"default": [], "items": {"type": "integer"}, "title": "Friends", "type": "array"}}, "required": ["id", "username", "password", "confirm_password"], "title": "User1", "type": "object"}'

In [50]:
user.model_json_schema()

{'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'username': {'title': 'Username', 'type': 'string'},
  'password': {'title': 'Password', 'type': 'string'},
  'confirm_password': {'title': 'Confirm Password', 'type': 'string'},
  'timestamp': {'anyOf': [{'format': 'date-time', 'type': 'string'},
    {'type': 'null'}],
   'default': None,
   'title': 'Timestamp'},
  'friends': {'default': [],
   'items': {'type': 'integer'},
   'title': 'Friends',
   'type': 'array'}},
 'required': ['id', 'username', 'password', 'confirm_password'],
 'title': 'User1',
 'type': 'object'}

## 5. 타입 처리하기 

## Constrained types

- Constrained Type을 통해 자신의 제한을 적용할 수 있습니다

In [53]:
from pydantic import (
    BaseModel,
    NegativeInt,
    PositiveInt,
    conint,
    conlist,
    constr
)


In [56]:

class Model(BaseModel):
    # minimum length of 2 and maximum length of 10
    short_str: constr(min_length=2, max_length=10)

    # remove whitespace from string
    strip_str: constr(strip_whitespace=True)

    # value must be greater than 1000 and less than 1024
    big_int: conint(gt=1000, lt=1024)
    
    # value is multiple of 5
    mod_int: conint(multiple_of=5)
    
    # must be a positive integer
    pos_int: PositiveInt
    
    # must be a negative integer
    neg_int: NegativeInt

    # list of integers that contains 1 to 4 items
    short_list: conlist(int)

## Strict types

- 검증된 값이 해당 유형이거나 해당 유형의 하위 유형인 경우에만 유효성 검사를 통과하는 엄격한 제한을 찾고 있다면 다음과 같은 엄격한 유형을 사용할 수 있습니다.

- StrictStr
- StrictInt
- StrictFloat
- StrictBool

In [57]:
from pydantic import BaseModel, StrictBool

In [58]:
class StrictBoolModel(BaseModel):
    strict_bool: StrictBool